# MAT281 - Laboratorio N°11

<a id='p1'></a>
## I.- Problema 01

Lista de actos delictivos registrados por el Service de police de la Ville de Montréal (SPVM).


<img src="http://henriquecapriles.com/wp-content/uploads/2017/02/femina_detenida-1080x675.jpg" width="480" height="360" align="center"/>

El conjunto de datos en estudio `interventionscitoyendo.csv` corresponde a  todos los delitos entre 2015 y agosto de 2020en Montreal. Cada delito está asociado en grandes categorías, y hay información sobre la ubicación, el momento del día, etc.

> **Nota**: Para más información seguir el siguiente el [link](https://donnees.montreal.ca/ville-de-montreal/actes-criminels).

In [ ]:
# librerias 

import os
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from metrics_regression import *


# graficos incrustados
plt.style.use('fivethirtyeight')
%matplotlib inline

# parametros esteticos de seaborn
sns.set_palette("deep", desat=.6)
sns.set_context(rc={"figure.figsize": (12, 4)})

In [ ]:
# read data

validate_categorie = [
  'Introduction', 'Méfait','Vol dans / sur véhicule à moteur', 'Vol de véhicule à moteur',
]

df = pd.read_csv(os.path.join("data","interventionscitoyendo.csv"), sep=",", encoding='latin-1')
df.columns = df.columns.str.lower()
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

df = df.loc[lambda x: x['categorie'].isin(validate_categorie)]
df = df.sort_values(['categorie','date'])
df.head()

Como tenemos muchos datos por categoría a nivel de día, agruparemos a nivel de **semanas** y separaremos cada serie temporal.

In [ ]:
cols = ['date','pdq']
y_s1 = df.loc[lambda x: x.categorie == validate_categorie[0] ][cols].set_index('date').resample('W').mean()
y_s2 = df.loc[lambda x: x.categorie == validate_categorie[1] ][cols].set_index('date').resample('W').mean()
y_s3 = df.loc[lambda x: x.categorie == validate_categorie[2] ][cols].set_index('date').resample('W').mean()
y_s4 = df.loc[lambda x: x.categorie == validate_categorie[3] ][cols].set_index('date').resample('W').mean()

El objetivo de este laboratorio es poder realizar un análisis completo del conjunto de datos en estudio, para eso debe responder las siguientes preguntas:

1. Realizar un gráfico para cada serie temporal $y\_{si}, i =1,2,3,4$.


In [ ]:
y_s1.plot(figsize=(15,3), color = 'r')
y_s2.plot(figsize=(15,3), color = 'r')
y_s3.plot(figsize=(15,3), color = 'r')
y_s4.plot(figsize=(15,3), color = 'r')

2. Escoger alguna serie temporal $y\_{si}, i =1,2,3,4$. Luego:

* Realice un análisis exploratorio de la serie temporal escogida
* Aplicar el modelo de pronóstico $SARIMA(p,d,q)x(P,D,Q,S)$, probando varias configuraciones de los hiperparámetros. Encuentre la mejor configuración. Concluya.
* Para el mejor modelo encontrado, verificar si el residuo corresponde a un ruido blanco.

> **Hint**: Tome como `target_date` =  '2021-01-01'. Recuerde considerar que su columna de valores se llama `pdq`.


In [ ]:
# se elige y_s4 
y = y_s4
display(y.head())
display(y.describe())

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
sns.boxplot(y.index.year, y.pdq, ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
sns.boxplot(y.index.month, y.pdq, ax=ax)
plt.show()

In [ ]:
# creando clase SarimaModels

class SarimaModels:
    def __init__(self,params):

        self.params = params
        
        
    @property
    def name_model(self):
        return f"SARIMA_{self.params[0]}X{self.params[1]}".replace(' ','')
    
    @staticmethod
    def test_train_model(y,date):
        mask_ds = y.index < date

        y_train = y[mask_ds]
        y_test = y[~mask_ds]        
        
        return y_train, y_test
    
    def fit_model(self,y,date):
        y_train, y_test = self.test_train_model(y,date )
        model = SARIMAX(y_train,
                        order=self.params[0],
                        seasonal_order=self.params[1],
                        enforce_stationarity=False,
                        enforce_invertibility=False)
        
        model_fit = model.fit(disp=0)

        return model_fit
    
    def df_testig(self,y,date):
        y_train, y_test = self.test_train_model(y,date )
        model = SARIMAX(y_train,
                        order=self.params[0],
                        seasonal_order=self.params[1],
                        enforce_stationarity=False,
                        enforce_invertibility=False)
        
        model_fit = model.fit(disp=0)
        
        start_index = y_test.index.min()
        end_index = y_test.index.max()

        preds = model_fit.get_prediction(start=start_index,end=end_index, dynamic=False)
        df_temp = pd.DataFrame(
            {
                'y':y_test['pdq'],
                'yhat': preds.predicted_mean
            }
        )
        
        return df_temp
    
    def metrics(self,y,date):
        df_temp = self.df_testig(y,date)
        df_metrics = summary_metrics(df_temp)
        df_metrics['model'] = self.name_model
        
        return df_metrics

# definir parametros 

import itertools

p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

params = list(itertools.product(pdq,seasonal_pdq))
target_date = '2021-01-01'

In [ ]:
# se procede a usar SARIMA(...)
target_date = '2021-01-01'

# crear conjuntos de entrenamiento y testeo
mask_ds = y.index < target_date

y_train = y[mask_ds]
y_test = y[~mask_ds]

# graficar
y_train['pdq'].plot()
y_test['pdq'].plot()
plt.show()

In [ ]:
# iterar sobre distintos escenarios, es decir distintos parametros para SARIMA

frames = []
for param in params:
    try:
        sarima_model = SarimaModels(param)
        df_metrics = sarima_model.metrics(y,target_date)
        frames.append(df_metrics)
    except:
        pass

In [ ]:
df_metrics_result = pd.concat(frames)
df_metrics_result.sort_values(['mae','mape'])

Los parámetros que lograron menores **mae** y **mape** son **S A R I M A (1,1,0)X(1,0,1,12)** y **S A R I M A (0,1,0)X(1,1,0,12)** y serán los que se ocuparan para ajustar el modelo.

In [ ]:
# ajustar 
# ajustar mejor modelo

param = [(1,1,0),(1,1,0,12)]
sarima_model =  SarimaModels(param)
model_fit = sarima_model.fit_model(y,target_date)
best_model = sarima_model.df_testig(y,target_date)
best_model.head()

In [ ]:
# graficar

preds = best_model['yhat']
ax = y['2019':].plot(label='observed')
preds.plot(ax=ax, label='Forecast', alpha=.7, figsize=(14, 7))
ax.set_xlabel('Date')
ax.set_ylabel('pdq')
plt.legend()
plt.show()


In [ ]:
# ajustar 
# ajustar mejor modelo

param = [(0,1,0),(1,1,0,12)]
sarima_model =  SarimaModels(param)
model_fit = sarima_model.fit_model(y,target_date)
best_model = sarima_model.df_testig(y,target_date)
best_model.head()
# graficar

preds = best_model['yhat']
ax = y['2019':].plot(label='observed') # se grafica desde el 2019 para observar mas claramente la forma de la serie teorica
preds.plot(ax=ax, label='Forecast', alpha=.7, figsize=(14, 7))
ax.set_xlabel('Date')
ax.set_ylabel('pdq')
plt.legend()
plt.show()

Con los parámetros **S A R I M A (1,1,0)X(1,0,1,12)** no se ajusta tan apropiadamente a los datos aunque si sigue la tendencia sinusoidal decreciente que tiene la serie en el año 2021. El modelo para los parámetros **S A R I M A (0,1,0)X(1,1,0,12)** parece ajustarse mejor, se le da preferencia a modelo con estos últimos parámetros.

In [ ]:
# errores
model_fit.plot_diagnostics(figsize=(16,8))
plt.show()

Del gráfico de la **residual estándarizada**, se nota que la serie es estacionaria, podría ser ruido blanco.

Desde el **histograma mas densidad estimada**, la distribución del error se parece a la distribución normal (0,1). Esto da mas pistas para asumir que la serie puede ser ruido blanco.

Para el scatter de **Quantiles de la muestra vs Quantiles teóricos** se aprecia una tendencia lineal por lo que las distribuciones de muestra y prueba se parecen.

En el **correlograma** no se entendió muy bien como leer este tipo de gráficos, pero se deduce que si los valores no tienden a 1 o -1 en el eje de las ordenadas, al parecer no hay correlación entre las variables, es decir serían independientes entre sí.